<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-c422-dipti/Exercises/day-12/Prompt_Production/Prompt_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Exercises: Integrating Prompts \& Measuring Prompt Performance


***

### Setup:

- Use OpenAI API or any language model API accessible in your Colab.
- If API keys are needed, ensure they are safely added as environment variables or input by the user.

In [1]:
from google.colab import userdata
import os

# Set your OpenAI API key securely in Colab Secrets (once)
# userdata.set("OPENAI_API_KEY", "your-api-key-here")

# Retrieve key in your notebook
openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("✅ OpenAI API key loaded safely")
else:
    print("❌ OpenAI API key not found. Please set it using Colab Secrets.")

✅ OpenAI API key loaded safely


In [2]:

!pip install --quiet openai -q
# Create client
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [3]:
# Helper Function to Send Prompts
def generate_response(prompt, model="gpt-4o-mini", temperature=0.7):
    """
    Send a prompt to the OpenAI model and return the response text.
    """
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"


***

# Exercise 1: Integrating Prompts in Applications and Pipelines

**Objective:** Learn how to integrate prompts into application workflows with versioning, automated testing, and CI/CD principles.

**Tasks:**

1. **Simulate prompt versioning:**
    - Create two versions of a prompt for a customer support chatbot (e.g., one generic, one with explicit instructions).
    - Store them as Python strings and switch between them in your code.

In [4]:
prompt_v1 = "Answer customer queries about returns."
prompt_v2 = "You are a helpful assistant answering customer return questions clearly and politely."

# Use either version
prompt = prompt_v2

response = generate_response(prompt + "\nCustomer: How do I return a product?")
print(response)

Thank you for reaching out! To return a product, please follow these steps:

1. **Check the Return Policy**: First, review our return policy to ensure your item is eligible for a return. 

2. **Prepare the Item**: Make sure the item is in its original condition, including packaging, tags, and accessories. 

3. **Request a Return Authorization**: If required, please contact our customer service team or log into your account to request a return authorization.

4. **Package the Item**: Carefully pack the item to prevent damage during shipping. 

5. **Ship the Item**: Use the return shipping label provided (if applicable) to send the item back to us. Make sure to keep a copy of the tracking number for your records.

6. **Refund Process**: Once we receive the returned item, we’ll process your refund according to our policy.

If you have any specific questions or need assistance with the return process, feel free to ask!


2. **Build a simple automated prompt test:**
    - Write a small function that runs a set of input prompts through your prompt versions.
    - Checks if responses include key expected phrases (e.g., "return policy", "contact support").
    - Output pass/fail for each test.

In [5]:
test_inputs = [
    "How do I return my purchase?",
    "What is the refund policy?"
]
expected_keywords = ["return policy", "contact support"]

def test_prompt(prompt):
    print(f"Testing prompt:\n{prompt}\n")
    for input_text in test_inputs:
        response = generate_response(prompt + "\nCustomer: " + input_text)
        print(f"Input: {input_text}\nResponse: {response}\n")
        passed = all(keyword in response.lower() for keyword in expected_keywords)
        print("Pass" if passed else "Fail", "\n")

# Test both versions
test_prompt(prompt_v1)
test_prompt(prompt_v2)

Testing prompt:
Answer customer queries about returns.

Input: How do I return my purchase?
Response: To return your purchase, please follow these steps:

1. **Review the Return Policy**: Check our return policy to ensure your item is eligible for a return. Items typically need to be returned within a specified timeframe and in their original condition.

2. **Prepare Your Item**: Repack the item securely in its original packaging, if possible. Make sure to include all accessories, manuals, and any other items that came with it.

3. **Obtain a Return Authorization**: Depending on our policy, you may need to request a return authorization. You can usually do this through our website or by contacting customer service.

4. **Print the Return Label**: If a return label is provided, print it out. If you're responsible for shipping costs, you can choose your preferred shipping method.

5. **Ship the Item**: Attach the return label to the package and ship it to the designated return address. K

3. **Discuss integration with CI/CD:**
    - Outline how this kind of automated prompt testing can be triggered on every prompt update in a CI/CD pipeline to ensure prompt quality before deployment.
    - Optionally, simulate prompt update by changing the prompt string and re-running tests.

***

# Exercise 2: Prompt Performance Measurement and Evaluation

**Objective:** Measure prompt quality using key metrics and understand how to evaluate and improve prompts systematically.

**Tasks:**

1. **Define and implement simple metrics:**
    - For each prompt response, measure:
        - **Relevance**: Does the response include certain keywords or meet a relevance criterion?
        - **Length**: Count the number of words or tokens.
        - **Consistency**: Run the same prompt multiple times and compare responses for similarity (string match or cosine similarity using embeddings if available).

Example:

In [7]:
import difflib

prompt = "Explain the benefits of renewable energy."

# Run same prompt 3 times
responses = [generate_response(prompt) for _ in range(3)]

print("Responses:")
for i, resp in enumerate(responses):
    print(f"Run {i+1}: {resp}\n")

# Simple consistency check: ratio of similarity between runs 1 and 2
similarity = difflib.SequenceMatcher(None, responses[0], responses[1]).ratio()
print(f"Similarity between first two runs: {similarity:.2f}")

Responses:
Run 1: Renewable energy offers a wide range of benefits that can positively impact the environment, economy, and society. Here are some key advantages:

1. **Environmental Benefits**:
   - **Reduced Greenhouse Gas Emissions**: Renewable energy sources, such as solar, wind, and hydro, produce little to no greenhouse gases during operation. This helps combat climate change and reduces air pollution.
   - **Lower Air and Water Pollution**: By minimizing reliance on fossil fuels, renewable energy reduces harmful emissions that contribute to air and water pollution, leading to improved public health and ecosystems.
   - **Sustainable Resource Use**: Renewable energy sources are naturally replenished, ensuring a sustainable and long-term energy supply without depleting natural resources.

2. **Economic Benefits**:
   - **Job Creation**: The renewable energy sector is labor-intensive, which means it creates a significant number of jobs in manufacturing, installation, maintenance, a

2. **Automated evaluation function:**
    - Build a function that takes a prompt and expected keywords (or a reference text) and computes pass/fail or similarity score.

In [8]:
def evaluate_prompt(prompt, expected_keywords):
    response = generate_response(prompt)
    print(f"Response:\n{response}\n")
    relevance = all(keyword.lower() in response.lower() for keyword in expected_keywords)
    length = len(response.split())
    print(f"Relevance: {'Passed' if relevance else 'Failed'}")
    print(f"Response length (words): {length}\n")

evaluate_prompt("Explain the importance of water conservation.", ["water", "conservation", "importance"])

Response:
Water conservation is crucial for several reasons, reflecting its significance for the environment, economy, and society as a whole. Here are some key points emphasizing its importance:

### 1. **Sustainable Resource Management**
- **Finite Resource**: Freshwater is a limited resource, with only about 2.5% of the Earth's water being freshwater, and a significant portion of that is not easily accessible.
- **Growing Demand**: As global populations increase and industrial and agricultural activities expand, the demand for freshwater grows, making conservation vital to ensure its availability for future generations.

### 2. **Environmental Protection**
- **Ecosystem Health**: Water conservation helps maintain healthy ecosystems. Aquatic habitats, wetlands, and rivers depend on sufficient water levels to support biodiversity.
- **Climate Resilience**: Conserving water can help mitigate the effects of droughts and climate change, which can lead to water scarcity and negatively imp

3. **Incorporate user feedback (simulated):**
    - Ask students to simulate or collect simple user feedback scores (e.g., 1 to 5 rating).
    - Show how this qualitative data complements automated metrics.
4. **Discuss trade-offs:**
    - Latency vs. response quality
    - Length vs. clarity
    - Consistency vs. creativity

***

## Summary for Students:

- Practice managing prompt versions and testing integrations as mini automation.
- Use keyword checks and repeated runs for simple prompt effectiveness measurement.
- Understand that prompt evaluation involves both quantitative metrics and qualitative feedback.
- Consider embedding prompt evaluation into automated pipelines for continuous prompt improvement.

***